# RNN with NodeAnalysis


In [1]:
from __future__ import print_function

import collections

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

import numpy as np
import tensorflow as tf 
import random
import time

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors, Phrases

import arithmetic_datasets as ad

## Create Training Data

In [3]:
num_examples = 20000
rstart = 1
rend = 10000
n_input = 3
n_hidden = 512

x_train, y_train = ad.gen_data_mult(num_examples, rstart, rend)
x_test, y_test = ad.gen_data_mult(50, rstart, rend)
dictionary = np.unique(np.array(x_train).flatten())

## Keras Model

In [7]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# model.add(layers.Embedding(vocab_size, 64, input_length=n_input)) <-- replaced by w2v
# model.add(layers.Dense(64, input_shape=(3,10)))
# Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(128))
new_shape = (1, n_input)
# model.add(layers.Dense(64, input_shape=new_shape))


rnn_cell = tf.keras.layers.StackedRNNCells([tf.keras.layers.LSTMCell(n_hidden),tf.keras.layers.LSTMCell(n_hidden)])
# model.add(layers.RNN(rnn_cell, input_length=n_input))
model.add(layers.RNN(rnn_cell, input_shape=new_shape))
# Add a Dense layer with 10 units.
model.add(layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 512)               3155968   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 3,156,481
Trainable params: 3,156,481
Non-trainable params: 0
_________________________________________________________________


## Prepare Batches

In [8]:
epochs = 50
batch_size_param = 1
batch_size = 1000*batch_size_param

print(x_train.shape)
print(y_train.shape)

model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # List of metrics to monitor
              metrics=['sparse_categorical_accuracy'])

print('# Fit model on training data')

(20000, 3)
(20000,)
# Fit model on training data


## Model Training

In [9]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size_param,
                    epochs= epochs)
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(x_val, y_val),

Epoch 1/50


ValueError: in user code:

    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:885 __call__
        input_spec.assert_input_compatibility(self.input_spec, inputs,
    c:\python38\lib\site-packages\tenso

## Show Test Results

In [ ]:
predictions = model.predict(x_val)

for x in range(0,len(predictions)):
    prediction = predictions[x]
    max = 0
    for i in range(0,len(prediction)):
        if prediction[i] > prediction[max]:
            max = i
    print(f'input: { [reverse_dictt[x_val2[x][i]] for i in range(0,len(x_val[x]))] } pred: {reverse_dictt[max]} --- actual: {[reverse_dictt[y_val[x][i]] for i in range(0,len(y_val[x]))] }')


## Results

### Accuracy

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
# plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

### Loss


In [ ]:
plt.title('model loss')
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


## Duration


In [ ]:
end_time = time.time()
print("duration: ",elapsed(end_time - start_time))